In [1]:
from selenium import webdriver
import selenium
import time
import pymongo
import requests
from bs4 import BeautifulSoup
import time
import json

import numpy as np
import pandas as pd

In [2]:
mc = pymongo.MongoClient()  # Connect to the MongoDB server using default settings

In [6]:

db = mc['movies_metadata']  # Use (or create) a database called 'movies_metadata'
test = db['test']  # Use (or create) a collection called 'test'


In [7]:
movies = db['movies'] #Create a collection called movies

In [ ]:
movies_clean = db['movies_clean']

In [589]:
movies_clean_v2 = db['movies_clean_v2']

In [590]:
mc.database_names()

['admin', 'local', 'movies_metadata', 'nyt_dump', 'tweets']

In [591]:
db.collection_names()

['test', 'movies', 'movies_backup', 'movies_clean']

In [379]:
movies_backup = db['movies_backup']
movies_backup.insert_many(movies.find())

0

In [592]:
db.collection_names()

['test', 'movies', 'movies_backup', 'movies_clean']

In [9]:
len(list(movies.find()))
#2017 movies populate movies.find first 510 records
#moving on...

1583

In [30]:
#2016 movies:
browser = webdriver.Firefox()
i = 2
browser.get("https://www.the-numbers.com/United-States/movies/year/2016")
while (browser.find_element_by_css_selector("#page_filling_chart > center:nth-child(7) > table:nth-child(3) > tbody:nth-child(1) > tr:nth-child(" + str(i) + ") > td:nth-child(5)").text != '$0'):
    
    #If the worldwide revenue is a nonzero amount... then go to the direct webpage for the movie
    #Then store that webpage's html code in the pymongo collection
    #The data on the webpage is sorted by worldwide box office revenue in descending order, with all the zero amounts 
    #at the bottom 
    url = browser.find_element_by_css_selector("#page_filling_chart > center:nth-child(7) > table:nth-child(3) > tbody:nth-child(1) > tr:nth-child(" + str(i) + ") > td:nth-child(1) > b:nth-child(1) > a:nth-child(1)").get_attribute('href')
    # pause between HTTP requests
    browser.get(url)
    DELAY = 5 + 5 * np.random.random() # time to wait between HTTP requests
    time.sleep(DELAY)
    html = browser.page_source
    movies.insert_one({'url': url,
                     'ts': time.time(),
                     'html': html
                     })
    print(i)
    i += 1
    browser.get("https://www.the-numbers.com/United-States/movies/year/2016")



519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552


In [18]:
#2015 movies:
browser = webdriver.Firefox()
i = 2
browser.get("https://www.the-numbers.com/United-States/movies/year/2015")
while (browser.find_element_by_css_selector("#page_filling_chart > center:nth-child(7) > table:nth-child(3) > tbody:nth-child(1) > tr:nth-child(" + str(i) + ") > td:nth-child(5)").text != '$0'):
    time.sleep(2)
    #If the worldwide revenue is a nonzero amount... then go to the direct webpage for the movie
    #Then store that webpage's html code in the pymongo collection
    url = browser.find_element_by_css_selector("#page_filling_chart > center:nth-child(7) > table:nth-child(3) > tbody:nth-child(1) > tr:nth-child(" + str(i) + ") > td:nth-child(1) > b:nth-child(1) > a:nth-child(1)").get_attribute('href')
    # pause between HTTP requests
    browser.get(url)
    DELAY = 5 + 5 * np.random.random() # time to wait between HTTP requests
    time.sleep(DELAY)
    html = browser.page_source
    movies.insert_one({'url': url,
                     'ts': time.time(),
                     'html': html
                     })
    print(i)
    i += 1
    browser.get("https://www.the-numbers.com/United-States/movies/year/2015")


483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523


In [50]:
import sys
sys.getsizeof(list(movies.find()))

13904

In [476]:

for i in range(len(list(movies.find()))):
    single_movie_data = {}
    html = movies.find()[i]['html']
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.find('h1', attrs={"itemprop" : "name"}).text
    tables = soup.select('div#summary table')
    for table in tables:
        table_html = str(table)
        if 'Amount' in table_html:
            table_metrics = table
        if 'Production Countries:' in table_html :
            table_details = table
            break

    for row in table_metrics.select('tr'):
        if len(row.select('td')) == 3:
            cols = row.select('td')
            field_name = cols[0].text.replace('.', '')
            field_value = cols[2].text
            single_movie_data[field_name] = field_value

    for row in table_details.select('tr'):
        cols = row.select('td')
        field_name = cols[0].text
        field_value = cols[1].text
        single_movie_data[field_name] = field_value

    tables = soup.find_all('div', attrs={'class':'cast_new'})

    for table in tables:
        table_html = str(table)
        if 'Lead Ensemble' in table_html:
            table_cast = table
        if 'Production and Technical Credits' in table_html :
            table_crew = table
            break
    table_crew

    cast_lst = []
    for row in table_cast.select('tr'):
        cols = row.select('td')
        actor_name= cols[0].text
        cast_lst.append(actor_name)
    single_movie_data['lead_cast'] = cast_lst

    for row in table_crew.select('tr'):
        cols = row.select('td')
        field_name = cols[2].text.replace('.','')
        if field_name in single_movie_data.keys():
            field_value = cols[0].text
            single_movie_data[field_name].append(field_value)
        else:
            single_movie_data[field_name] = [cols[0].text]
    
    movies_clean.insert_one({'title': title,
                            'data': single_movie_data})
    print(i,title)

0 Star Wars Ep. VIII: The Last Jedi (2017)
1 Beauty and the Beast (2017)
2 The Fate of the Furious (2017)
3 Despicable Me 3 (2017)
4 Jumanji: Welcome to the Jungle (2017)
5 Spider-Man: Homecoming (2017)
6 Guardians of the Galaxy Vol 2 (2017)
7 Thor: Ragnarok (2017)
8 Wonder Woman (2017)
9 Coco (2017)
10 Pirates of the Caribbean: Dead Men Tell No Tales (2017)
11 It (2017)
12 Justice League (2017)
13 Logan (2017)
14 Transformers: The Last Knight (2017)
15 Kong: Skull Island (2017)
16 The Boss Baby (2017)
17 Dunkirk (2017)
18 War for the Planet of the Apes (2017)
19 The Greatest Showman (2017)
20 The Mummy (2017)
21 Kingsman: The Golden Circle (2017)
22 Cars 3 (2017)
23 Fifty Shades Darker (2017)
24 Murder on the Orient Express (2017)
25 xXx: Return of Xander Cage (2017)
26 The Lego Batman Movie (2017)
27 Annabelle: Creation (2017)
28 Wonder (2017)
29 Ferdinand (2017)
30 Split (2017)
31 Blade Runner 2049 (2017)
32 Get Out (2017)
33 Alien: Covenant (2017)
34 Baby Driver (2017)
35 Geostorm 

299 Restless Creature: Wendy Whelan (2017)
300 The Last Dalai Lama? (2017)
301 Adventure Time with Finn and Jake Islands (2017)
302 The Stoic (2017)
303 Hare Krishna! The Mantra, The Movement and the Swami Who Started it all (2017)
304 Dawson City: Frozen Time (2017)
305 The B-Side (2017)
306 Memento (2017)
307 We Love You, Sally Carmichael (2017)
308 Abacus: Small Enough to Jail (2017)
309 Warren Miller's Here, There & Everywhere (2017)
310 Newness (2017)
311 The Bachelors (2017)
312 Bullet Head (2017)
313 Aida’s Secrets (2017)
314 Tragedy Girls (2017)
315 Tim Timmerman: Hope of America (2017)
316 I Called Him Morgan (2017)
317 Dina (2017)
318 Double Play (2017)
319 Keep Watching (2017)
320 Because of Grácia (2017)
321 Chapter & Verse (2017)
322 Score: A Film Music Documentary (2017)
323 Bill Nye: Science Guy (2017)
324 First Kill (2017)
325 Wait For Your Laugh (2017)
326 Te Ata (2017)
327 Buster’s Mal Heart (2017)
328 Max 2: White House Hero (2017)
329 Turn it Around: The Story of Ea

588 The Boss (2016)
589 Manchester by the Sea (2016)
590 Kubo and the Two Strings (2016)
591 Mike and Dave Need Wedding Dates (2016)
592 Miracles from Heaven (2016)
593 Tyler Perry’s Boo! A Madea Halloween (2016)
594 13 Hours: The Secret Soldiers of Benghazi (2016)
595 The Boy (2016)
596 Moonlight (2016)
597 Fences (2016)
598 Hail, Caesar! (2016)
599 Monster Trucks (2016)
600 Nerve (2016)
601 The Nice Guys (2016)
602 Florence Foster Jenkins (2016)
603 Zoolander 2 (2016)
604 All Eyez on Me (2016)
605 Barbershop: The Next Cut (2016)
606 Patriots Day (2016)
607 Mother’s Day (2016)
608 The Finest Hours (2016)
609 A Monster Calls (2016)
610 Risen (2016)
611 Eddie the Eagle (2016)
612 Café Society (2016)
613 Almost Christmas (2016)
614 The Forest (2016)
615 The Witch (2016)
616 Crouching Tiger, Hidden Dragon: Sword of Destiny (2016)
617 Hell or High Water (2016)
618 Blair Witch (2016)
619 Nocturnal Animals (2016)
620 Masterminds (2016)
621 When the Bough Breaks (2016)
622 Keeping Up with the

876 In Pursuit of Silence (2016)
877 Of Mind and Music (2016)
878 Spa Night (2016)
879 Is That a Gun in Your Pocket? (2016)
880 Kanye West: The Global Premiere of WAVES (2016)
881 Catching the Sun (2016)
882 Troublemakers: The Story of Land Art (2016)
883 Germans and Jews (2016)
884 Mapplethorpe: Look at the Pictures (2016)
885 Restored Me (2016)
886 How to Let Go of the World and Love All The Things Climate Can’t Change (2016)
887 Kill Me, Deadly (2016)
888 Who Gets the Dog? (2016)
889 Full of Grace (2016)
890 The Homeless Billionaire (2016)
891 Off the Rails (2016)
892 The Intervention (2016)
893 The Curse of Sleeping Beauty (2016)
894 Jack of the Red Hearts (2016)
895 Lamb (2016)
896 Being Charlie (2016)
897 The Seventh Fire (2016)
898 The Best Democracy Money Can Buy (2016)
899 The Unknowns (2016)
900 3 Weeks in Yerevan (2016)
901 Time to Choose (2016)
902 The Eyes of My Mother (2016)
903 Take Me to the River (2016)
904 Kate Plays Christine (2016)
905 Unlocking the Cage (2016)
906 

1161 Criminal (2015)
1162 Room (2015)
1163 Steve Jobs (2015)
1164 The Lazarus Effect (2015)
1165 A Walk in the Woods (2015)
1166 Snowden (2015)
1167 Self/Less (2015)
1168 The Woman in Black 2: Angel of Death (2015)
1169 Victor Frankenstein (2015)
1170 Project Almanac (2015)
1171 Burnt (2015)
1172 Secret in Their Eyes (2015)
1173 Mortdecai (2015)
1174 American Ultra (2015)
1175 Far from the Madding Crowd (2015)
1176 Mr. Holmes (2015)
1177 The 33 (2015)
1178 Irrational Man (2015)
1179 Aloha (2015)
1180 It Follows (2015)
1181 Solace (2015)
1182 Black or White (2015)
1183 Blackhat (2015)
1184 Dope (2015)
1185 Little Boy (2015)
1186 Monkey Kingdom (2015)
1187 Scouts Guide to the Zombie Apocalypse (2015)
1188 The Vatican Tapes (2015)
1189 Love & Mercy (2015)
1190 Woodlawn (2015)
1191 Do You Believe? (2015)
1192 The Gunman (2015)
1193 While We're Young (2015)
1194 Strange Magic (2015)
1195 Macbeth (2015)
1196 The Green Inferno (2015)
1197 Unfinished Business (2015)
1198 Hot Tub Time Machine 2

1445 Prophet’s Prey (2015)
1446 Hungry Hearts (2015)
1447 Lila & Eve (2015)
1448 Welcome to Leith (2015)
1449 Finders Keepers (2015)
1450 Speak No Evil (2015)
1451 Black November (2015)
1452 Amira & Sam (2015)
1453 Don Verdean (2015)
1454 The Winding Stream (2015)
1455 The Nightmare (2015)
1456 Match (2015)
1457 Runoff (2015)
1458 Paul Taylor Creative Domain (2015)
1459 3 and 1/2 Minutes (2015)
1460 Americons (2015)
1461 Look At Us Now, Mother! (2015)
1462 Like Sunday, Like Rain (2015)
1463 7 Chinese Brothers (2015)
1464 Sembene! (2015)
1465 Tap World (2015)
1466 Peace Officer (2015)
1467 Preggoland (2015)
1468 Out of the Dark (2015)
1469 Backstreet Boys: Show 'Em What You're Made Of (2015)
1470 Kill Me Three Times (2015)
1471 The Diabolical (2015)
1472 Slow Learners (2015)
1473 Can't Stand Losing You: Surviving the Police (2015)
1474 Animals (2015)
1475 Boy Meets Girl (2015)
1476 Friends and Romans (2015)
1477 The Runner (2015)
1478 Misery Loves Comedy (2015)
1479 This Changes Everyth

In [478]:
len(list(movies.find()))

1583

In [477]:
len(list(movies_clean.find()))

1583

In [493]:
df = pd.DataFrame(list(movies_clean.find()))
df.head()

,_id,data,title
0,5af4ae197cfdd30e191994b0,"{'All Time Domestic Box Office': '$620,181,382...",Star Wars Ep. VIII: The Last Jedi (2017)
1,5af4ae197cfdd30e191994b1,"{'All Time Domestic Box Office': '$504,014,165...",Beauty and the Beast (2017)
2,5af4ae197cfdd30e191994b2,{'All Time Domestic Box Office (Rank 101-200)'...,The Fate of the Furious (2017)
3,5af4ae197cfdd30e191994b3,"{'All Time Domestic Box Office': '$264,624,300...",Despicable Me 3 (2017)
4,5af4ae1a7cfdd30e191994b4,"{'All Time Domestic Box Office': '$404,353,892...",Jumanji: Welcome to the Jungle (2017)


In [501]:
df2 = pd.DataFrame(list(df['data']))

In [534]:
df2.columns[750:850]

Index(['All Time Worldwide Box Office (Rank 8,401-8,500)',
       'All Time Worldwide Box Office (Rank 8,501-8,600)',
       'All Time Worldwide Box Office (Rank 8,601-8,700)',
       'All Time Worldwide Box Office (Rank 8,701-8,800)',
       'All Time Worldwide Box Office (Rank 8,801-8,900)',
       'All Time Worldwide Box Office (Rank 8,901-9,000)',
       'All Time Worldwide Box Office (Rank 801-900)',
       'All Time Worldwide Box Office (Rank 9,001-9,100)',
       'All Time Worldwide Box Office (Rank 9,101-9,200)',
       'All Time Worldwide Box Office (Rank 9,201-9,300)',
       'All Time Worldwide Box Office (Rank 9,301-9,400)',
       'All Time Worldwide Box Office (Rank 9,401-9,500)',
       'All Time Worldwide Box Office (Rank 9,501-9,600)',
       'All Time Worldwide Box Office (Rank 9,601-9,700)',
       'All Time Worldwide Box Office (Rank 9,701-9,800)',
       'All Time Worldwide Box Office (Rank 9,801-9,900)',
       'All Time Worldwide Box Office (Rank 9,901-10,000)',


In [535]:
df3 = df2.iloc[:, 557:600]
df3

,All Time Worldwide Box Office,"All Time Worldwide Box Office (Rank 1,001-1,100)","All Time Worldwide Box Office (Rank 1,101-1,200)","All Time Worldwide Box Office (Rank 1,201-1,300)","All Time Worldwide Box Office (Rank 1,301-1,400)","All Time Worldwide Box Office (Rank 1,401-1,500)","All Time Worldwide Box Office (Rank 1,501-1,600)","All Time Worldwide Box Office (Rank 1,601-1,700)","All Time Worldwide Box Office (Rank 1,701-1,800)","All Time Worldwide Box Office (Rank 1,801-1,900)",...,"All Time Worldwide Box Office (Rank 12,101-12,200)","All Time Worldwide Box Office (Rank 12,201-12,300)","All Time Worldwide Box Office (Rank 12,301-12,400)","All Time Worldwide Box Office (Rank 12,401-12,500)","All Time Worldwide Box Office (Rank 12,501-12,600)","All Time Worldwide Box Office (Rank 12,601-12,700)","All Time Worldwide Box Office (Rank 12,701-12,800)","All Time Worldwide Box Office (Rank 12,801-12,900)","All Time Worldwide Box Office (Rank 12,901-13,000)","All Time Worldwide Box Office (Rank 13,001-13,100)"
0,"$1,320,214,241",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"$1,263,109,573",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"$1,237,466,026",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"$1,034,542,646",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"$958,513,171",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"$880,210,429",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"$863,197,744",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"$850,653,412",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"$817,012,815",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"$800,398,408",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [519]:
'All Time Worldwide Box Office' in df2.columns


True

In [ ]:
# details_df.to_json(orient='records')

In [483]:
import sys
sys.getsizeof((movies.find()))



56

In [661]:
%%time
#V2

for i in range(len(list(movies.find()))):
    single_movie_data = {}
    html = movies.find()[i]['html']
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.find('h1', attrs={"itemprop" : "name"}).text


    table = soup.find('table', attrs={'id':'movie_finances'})
    for row in table.select('tr'):
        cols = row.select('td')
        if len(cols) > 1:
            field_name = cols[0].text.replace('.', '')
            field_value = cols[1].text.replace('$', '').replace(',', '')
        single_movie_data[field_name] = field_value

    tables = soup.select('div#summary table')
    for table in tables:
        table_html = str(table)
        if 'Production Countries:' in table_html :
            table_details = table
            break

    for row in table_details.select('tr'):
        cols = row.select('td')
        field_name = cols[0].text.replace(':', '').replace('\xa0', ' ')
        field_value = cols[1].text.replace('$', '').replace(',', '')
        single_movie_data[field_name] = field_value

    tables = soup.find_all('div', attrs={'class':'cast_new'})

    for table in tables:
        table_html = str(table)
        if 'Lead Ensemble' in table_html:
            table_cast = table
        if 'Production and Technical Credits' in table_html :
            table_crew = table
            break

    cast_lst = []
    for row in table_cast.select('tr'):
        cols = row.select('td')
        actor_name= cols[0].text
        cast_lst.append(actor_name)
    single_movie_data['lead_cast'] = cast_lst

    for row in table_crew.select('tr'):
        cols = row.select('td')
        field_name = cols[2].text.replace('.','')
        if field_name in single_movie_data.keys():
            if type(single_movie_data[field_name]) == list:
                field_value = cols[0].text
                single_movie_data[field_name].append(field_value)
            else:
                field_value = cols[0].text
                single_movie_data[field_name] = [single_movie_data[field_name]]
                single_movie_data[field_name].append(field_value)
        else:
            single_movie_data[field_name] = cols[0].text

    movies_clean_v2.insert_one({'title': title,
                            'data': single_movie_data})
    print(i,title)


0 Star Wars Ep. VIII: The Last Jedi (2017)
1 Beauty and the Beast (2017)
2 The Fate of the Furious (2017)
3 Despicable Me 3 (2017)
4 Jumanji: Welcome to the Jungle (2017)
5 Spider-Man: Homecoming (2017)
6 Guardians of the Galaxy Vol 2 (2017)
7 Thor: Ragnarok (2017)
8 Wonder Woman (2017)
9 Coco (2017)
10 Pirates of the Caribbean: Dead Men Tell No Tales (2017)
11 It (2017)
12 Justice League (2017)
13 Logan (2017)
14 Transformers: The Last Knight (2017)
15 Kong: Skull Island (2017)
16 The Boss Baby (2017)
17 Dunkirk (2017)
18 War for the Planet of the Apes (2017)
19 The Greatest Showman (2017)
20 The Mummy (2017)
21 Kingsman: The Golden Circle (2017)
22 Cars 3 (2017)
23 Fifty Shades Darker (2017)
24 Murder on the Orient Express (2017)
25 xXx: Return of Xander Cage (2017)
26 The Lego Batman Movie (2017)
27 Annabelle: Creation (2017)
28 Wonder (2017)
29 Ferdinand (2017)
30 Split (2017)
31 Blade Runner 2049 (2017)
32 Get Out (2017)
33 Alien: Covenant (2017)
34 Baby Driver (2017)
35 Geostorm 

299 Restless Creature: Wendy Whelan (2017)
300 The Last Dalai Lama? (2017)
301 Adventure Time with Finn and Jake Islands (2017)
302 The Stoic (2017)
303 Hare Krishna! The Mantra, The Movement and the Swami Who Started it all (2017)
304 Dawson City: Frozen Time (2017)
305 The B-Side (2017)
306 Memento (2017)
307 We Love You, Sally Carmichael (2017)
308 Abacus: Small Enough to Jail (2017)
309 Warren Miller's Here, There & Everywhere (2017)
310 Newness (2017)
311 The Bachelors (2017)
312 Bullet Head (2017)
313 Aida’s Secrets (2017)
314 Tragedy Girls (2017)
315 Tim Timmerman: Hope of America (2017)
316 I Called Him Morgan (2017)
317 Dina (2017)
318 Double Play (2017)
319 Keep Watching (2017)
320 Because of Grácia (2017)
321 Chapter & Verse (2017)
322 Score: A Film Music Documentary (2017)
323 Bill Nye: Science Guy (2017)
324 First Kill (2017)
325 Wait For Your Laugh (2017)
326 Te Ata (2017)
327 Buster’s Mal Heart (2017)
328 Max 2: White House Hero (2017)
329 Turn it Around: The Story of Ea

587 Underworld: Blood Wars (2016)
588 The Boss (2016)
589 Manchester by the Sea (2016)
590 Kubo and the Two Strings (2016)
591 Mike and Dave Need Wedding Dates (2016)
592 Miracles from Heaven (2016)
593 Tyler Perry’s Boo! A Madea Halloween (2016)
594 13 Hours: The Secret Soldiers of Benghazi (2016)
595 The Boy (2016)
596 Moonlight (2016)
597 Fences (2016)
598 Hail, Caesar! (2016)
599 Monster Trucks (2016)
600 Nerve (2016)
601 The Nice Guys (2016)
602 Florence Foster Jenkins (2016)
603 Zoolander 2 (2016)
604 All Eyez on Me (2016)
605 Barbershop: The Next Cut (2016)
606 Patriots Day (2016)
607 Mother’s Day (2016)
608 The Finest Hours (2016)
609 A Monster Calls (2016)
610 Risen (2016)
611 Eddie the Eagle (2016)
612 Café Society (2016)
613 Almost Christmas (2016)
614 The Forest (2016)
615 The Witch (2016)
616 Crouching Tiger, Hidden Dragon: Sword of Destiny (2016)
617 Hell or High Water (2016)
618 Blair Witch (2016)
619 Nocturnal Animals (2016)
620 Masterminds (2016)
621 When the Bough Bre

877 Of Mind and Music (2016)
878 Spa Night (2016)
879 Is That a Gun in Your Pocket? (2016)
880 Kanye West: The Global Premiere of WAVES (2016)
881 Catching the Sun (2016)
882 Troublemakers: The Story of Land Art (2016)
883 Germans and Jews (2016)
884 Mapplethorpe: Look at the Pictures (2016)
885 Restored Me (2016)
886 How to Let Go of the World and Love All The Things Climate Can’t Change (2016)
887 Kill Me, Deadly (2016)
888 Who Gets the Dog? (2016)
889 Full of Grace (2016)
890 The Homeless Billionaire (2016)
891 Off the Rails (2016)
892 The Intervention (2016)
893 The Curse of Sleeping Beauty (2016)
894 Jack of the Red Hearts (2016)
895 Lamb (2016)
896 Being Charlie (2016)
897 The Seventh Fire (2016)
898 The Best Democracy Money Can Buy (2016)
899 The Unknowns (2016)
900 3 Weeks in Yerevan (2016)
901 Time to Choose (2016)
902 The Eyes of My Mother (2016)
903 Take Me to the River (2016)
904 Kate Plays Christine (2016)
905 Unlocking the Cage (2016)
906 Anesthesia (2016)
907 Speed Siste

1164 The Lazarus Effect (2015)
1165 A Walk in the Woods (2015)
1166 Snowden (2015)
1167 Self/Less (2015)
1168 The Woman in Black 2: Angel of Death (2015)
1169 Victor Frankenstein (2015)
1170 Project Almanac (2015)
1171 Burnt (2015)
1172 Secret in Their Eyes (2015)
1173 Mortdecai (2015)
1174 American Ultra (2015)
1175 Far from the Madding Crowd (2015)
1176 Mr. Holmes (2015)
1177 The 33 (2015)
1178 Irrational Man (2015)
1179 Aloha (2015)
1180 It Follows (2015)
1181 Solace (2015)
1182 Black or White (2015)
1183 Blackhat (2015)
1184 Dope (2015)
1185 Little Boy (2015)
1186 Monkey Kingdom (2015)
1187 Scouts Guide to the Zombie Apocalypse (2015)
1188 The Vatican Tapes (2015)
1189 Love & Mercy (2015)
1190 Woodlawn (2015)
1191 Do You Believe? (2015)
1192 The Gunman (2015)
1193 While We're Young (2015)
1194 Strange Magic (2015)
1195 Macbeth (2015)
1196 The Green Inferno (2015)
1197 Unfinished Business (2015)
1198 Hot Tub Time Machine 2 (2015)
1199 Trumbo (2015)
1200 We Are Your Friends (2015)
12

1445 Prophet’s Prey (2015)
1446 Hungry Hearts (2015)
1447 Lila & Eve (2015)
1448 Welcome to Leith (2015)
1449 Finders Keepers (2015)
1450 Speak No Evil (2015)
1451 Black November (2015)
1452 Amira & Sam (2015)
1453 Don Verdean (2015)
1454 The Winding Stream (2015)
1455 The Nightmare (2015)
1456 Match (2015)
1457 Runoff (2015)
1458 Paul Taylor Creative Domain (2015)
1459 3 and 1/2 Minutes (2015)
1460 Americons (2015)
1461 Look At Us Now, Mother! (2015)
1462 Like Sunday, Like Rain (2015)
1463 7 Chinese Brothers (2015)
1464 Sembene! (2015)
1465 Tap World (2015)
1466 Peace Officer (2015)
1467 Preggoland (2015)
1468 Out of the Dark (2015)
1469 Backstreet Boys: Show 'Em What You're Made Of (2015)
1470 Kill Me Three Times (2015)
1471 The Diabolical (2015)
1472 Slow Learners (2015)
1473 Can't Stand Losing You: Surviving the Police (2015)
1474 Animals (2015)
1475 Boy Meets Girl (2015)
1476 Friends and Romans (2015)
1477 The Runner (2015)
1478 Misery Loves Comedy (2015)
1479 This Changes Everyth

In [659]:
# movies_clean_v2.delete_many({})


In [662]:
len(list(movies_clean_v2.find()))

1583

In [663]:
df_clean = pd.DataFrame(list(movies_clean_v2.find()))
df_clean.head()

,_id,data,title
0,5af4cd0d7cfdd30e1919a126,"{'Casting Director': 'Nick Horbaczewski', 'Dom...",Star Wars Ep. VIII: The Last Jedi (2017)
1,5af4cd0d7cfdd30e1919a127,{'Visual Effects Supervisor': 'Chris Corbould'...,Beauty and the Beast (2017)
2,5af4cd0d7cfdd30e1919a128,"{'Set Decorator': ['Tim Rice', 'Danielle Berma...",The Fate of the Furious (2017)
3,5af4cd0d7cfdd30e1919a129,"{'Second Unit Director': 'Jack Gill', 'Domesti...",Despicable Me 3 (2017)
4,5af4cd0e7cfdd30e1919a12a,"{'Score Mixer': 'Noah Snyder', 'Domestic Box O...",Jumanji: Welcome to the Jungle (2017)


In [664]:
df_clean_data = pd.DataFrame(list(df_clean['data']))
df_clean_data.head()

,,1st Assistant Director,3-D and Digital Image Producer,3D Animator,3D Stereoscopic Supervisor,ADR Editor,ADR Mixer,ADR Recording Engineer,ADR/Dialogue Editor,Action Choreographer,...,Visual Effects Supervisor: MPC,Visual Effects Supervisor: Milk Visual Effects,Visual Effects by,Wardrobe Supervisor,Worldwide Box Office,Written By,Written by,based on the novel “A Monster Calls” by,lead_cast,story based on a novel by
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1320214241,NaN,NaN,NaN,"[Mark Hamill, Carrie Fisher, Adam Driver, Dais...",NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1263109573,NaN,NaN,NaN,"[Emma Watson, Dan Stevens, Luke Evans, Kevin K...",NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1237466026,NaN,NaN,NaN,"[Vin Diesel, Dwayne Johnson, Jason Statham, Mi...",NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1034542646,NaN,NaN,NaN,"[Vin Diesel, Dwayne Johnson, Jason Statham, Mi...",NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,958513171,NaN,NaN,NaN,"[Dwayne Johnson, Jack Black, Kevin Hart, Karen...",NaN


In [755]:
df_clean_data_trim = df_clean_data.loc[:, ['Worldwide Box Office', 'Domestic Box Office', 'International Box Office','Production Budget','Source']]
df_clean_data_trim

,Worldwide Box Office,Domestic Box Office,International Box Office,Production Budget,Source
0,1320214241,620181382,700032859,NaN,Original Screenplay
1,1263109573,504014165,759095408,160000000,Remake
2,1237466026,225764765,1011701261,250000000,Original Screenplay
3,1034542646,264624300,769918346,75000000,Original Screenplay
4,958513171,404353892,554159279,90000000,Original Screenplay
5,880210429,334201140,546009289,175000000,Based on Comic/Graphic Novel
6,863197744,389813101,473384643,200000000,Based on Comic/Graphic Novel
7,850653412,315058289,535595123,180000000,Based on Comic/Graphic Novel
8,817012815,412563408,404449407,150000000,Based on Comic/Graphic Novel
9,800398408,209726015,590672393,175000000,Based on Folk Tale/Legend/Fairytale


In [682]:
df_clean_data_trim['Domestic Box Office'].fillna(0, inplace=True)
df_clean_data_trim['Domestic Box Office']

0       620181382
1       504014165
2       225764765
3       264624300
4       404353892
5       334201140
6       389813101
7       315058289
8       412563408
9       209726015
10      172558876
11      327481748
12      229024295
13      226277068
14      130168683
15      168052812
16      175003033
17      190068280
18      146880162
19      173789109
20       80101125
21      100234838
22      152901115
23      114434010
24      102826543
25       44898413
26      175750384
27      102092201
28      132422809
29       84364229
          ...    
1553         3256
1554         3130
1555         2634
1556         2571
1557            0
1558            0
1559         2421
1560            0
1561         2105
1562         1800
1563         1903
1564         1747
1565         1711
1566            0
1567         1597
1568            0
1569         1484
1570            0
1571            0
1572            0
1573         1297
1574         1242
1575         1036
1576            0
1577      

In [756]:
# df_clean_data_trim['WW Aggregate'] = int(df_clean_data_trim['Domestic Box Office']) + int(df_clean_data_trim['International Box Office'])
# df_clean_data_trim

df_clean_data_trim['Domestic Box Office'].fillna(0, inplace=True)
df_clean_data_trim['International Box Office'].fillna(0, inplace=True)
df_clean_data_trim['Production Budget'].fillna(10000000, inplace=True)

df_clean_data_trim['Domestic Box Office'] = df_clean_data_trim['Domestic Box Office'].apply(pd.to_numeric)
df_clean_data_trim['International Box Office'] = df_clean_data_trim['International Box Office'].apply(pd.to_numeric)
df_clean_data_trim['ww_agg'] = df_clean_data_trim['Domestic Box Office'] + df_clean_data_trim['International Box Office']
df_clean_data_trim['Production Budget'] = df_clean_data_trim['Production Budget'].apply(pd.to_numeric)

df_clean_data_trim

,Worldwide Box Office,Domestic Box Office,International Box Office,Production Budget,Source,ww_agg
0,1320214241,620181382,700032859,10000000,Original Screenplay,1320214241
1,1263109573,504014165,759095408,160000000,Remake,1263109573
2,1237466026,225764765,1011701261,250000000,Original Screenplay,1237466026
3,1034542646,264624300,769918346,75000000,Original Screenplay,1034542646
4,958513171,404353892,554159279,90000000,Original Screenplay,958513171
5,880210429,334201140,546009289,175000000,Based on Comic/Graphic Novel,880210429
6,863197744,389813101,473384643,200000000,Based on Comic/Graphic Novel,863197744
7,850653412,315058289,535595123,180000000,Based on Comic/Graphic Novel,850653412
8,817012815,412563408,404449407,150000000,Based on Comic/Graphic Novel,817012815
9,800398408,209726015,590672393,175000000,Based on Folk Tale/Legend/Fairytale,800398408


In [723]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve,roc_auc_score

In [761]:
X = df_clean_data_trim.iloc[:,3:4]
y = df_clean_data_trim.iloc[:,5]

In [762]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [763]:
base_model = LinearRegression()

In [764]:
base_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [765]:
y_pred = base_model.predict(X_test)

In [766]:
base_model.score(X_test,y_test)

0.77236797510950528

In [767]:
base_model.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': 1, 'normalize': False}

In [768]:
df_clean_data_trim['log_ww_agg'] = df_clean_data_trim['ww_agg'].apply(np.log)

In [769]:
X = df_clean_data_trim.iloc[:,3:4]
y = df_clean_data_trim.iloc[:,6]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [770]:
base_model_logrev = LinearRegression()
base_model_logrev.fit(X_train, y_train)
y_pred_logrev = base_model_logrev.predict(X_test)
base_model_logrev.score(X_test,y_test)

0.29854408189690151

In [771]:
df_clean_data_trim['log_budget'] = df_clean_data_trim['Production Budget'].apply(np.log)

In [772]:
df_clean_data_trim.head()

,Worldwide Box Office,Domestic Box Office,International Box Office,Production Budget,Source,ww_agg,log_ww_agg,log_budget
0,1320214241,620181382,700032859,10000000,Original Screenplay,1320214241,21.001060,16.118096
1,1263109573,504014165,759095408,160000000,Remake,1263109573,20.956842,18.890684
2,1237466026,225764765,1011701261,250000000,Original Screenplay,1237466026,20.936332,19.336971
3,1034542646,264624300,769918346,75000000,Original Screenplay,1034542646,20.757225,18.132999
4,958513171,404353892,554159279,90000000,Original Screenplay,958513171,20.680894,18.315320


In [790]:
X = df_clean_data_trim.iloc[:,[7]]
y = df_clean_data_trim.iloc[:,6]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [787]:
base_model_logrev_2 = LinearRegression()
base_model_logrev_2.fit(X_train, y_train)
y_pred_logrev_2 = base_model_logrev_2.predict(X_test)
base_model_logrev_2.score(X_test,y_test)

0.25021551803196052

In [780]:
df_source_dummies = pd.concat( [df_clean_data_trim, pd.get_dummies(df_clean_data_trim['Source'])], axis=1)

In [781]:
df_source_dummies.head()

,Worldwide Box Office,Domestic Box Office,International Box Office,Production Budget,Source,ww_agg,log_ww_agg,log_budget,Based on Comic/Graphic Novel,Based on Factual Book/Article,...,Based on Religious Text,Based on Short Film,Based on TV,Based on Theme Park Ride,Based on Toy,Based on Web Series,Compilation,Original Screenplay,Remake,Spin-Off
0,1320214241,620181382,700032859,10000000,Original Screenplay,1320214241,21.001060,16.118096,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1263109573,504014165,759095408,160000000,Remake,1263109573,20.956842,18.890684,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1237466026,225764765,1011701261,250000000,Original Screenplay,1237466026,20.936332,19.336971,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1034542646,264624300,769918346,75000000,Original Screenplay,1034542646,20.757225,18.132999,0,0,...,0,0,0,0,0,0,0,1,0,0
4,958513171,404353892,554159279,90000000,Original Screenplay,958513171,20.680894,18.315320,0,0,...,0,0,0,0,0,0,0,1,0,0


In [791]:
X = df_source_dummies.iloc[:, 7:]
y = df_source_dummies.iloc[:,6]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [792]:
base_model_logrev_sourcedummies = LinearRegression()
base_model_logrev_sourcedummies.fit(X_train, y_train)
y_pred_logrev_sourcedummies = base_model_logrev_sourcedummies.predict(X_test)
base_model_logrev_sourcedummies.score(X_test,y_test)

0.34325894210271879

In [ ]:
metadata = pd.read_csv('data/movies_metadata.csv', low_memory= False)

In [ ]:
metadata.head()

In [5]:
len(metadata[metadata['revenue'] == 0])

38052

In [6]:
metadata[metadata['revenue'] == 0]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
11,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,12110,tt0112896,en,Dracula: Dead and Loving It,When a lawyer shows up at the vampire's doorst...,...,1995-12-22,0.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Dracula: Dead and Loving It,False,5.7,210.0
21,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,1710,tt0112722,en,Copycat,An agoraphobic psychologist and a female detec...,...,1995-10-27,0.0,124.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,One man is copying the most notorious killers ...,Copycat,False,6.5,199.0
23,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",NaN,12665,tt0114168,en,Powder,Harassed by classmates who won't accept his sh...,...,1995-10-27,0.0,111.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,An extraordinary encounter with another human ...,Powder,False,6.3,143.0
25,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,16420,tt0114057,en,Othello,The evil Iago pretends to be friend of Othello...,...,1995-12-15,0.0,123.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Envy, greed, jealousy and love.",Othello,False,7.0,33.0
27,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,17015,tt0114117,en,Persuasion,This film adaptation of Jane Austen's last nov...,...,1995-09-27,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Persuasion,False,7.4,36.0
29,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,37557,tt0115012,zh,摇啊摇，摇到外婆桥,A provincial boy related to a Shanghai crime f...,...,1995-04-30,0.0,108.0,"[{'iso_639_1': 'zh', 'name': '普通话'}]",Released,In 1930's Shanghai violence was not the proble...,Shanghai Triad,False,6.5,17.0
32,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 12, ...",NaN,78802,tt0114952,fr,"Guillaumet, les ailes du courage",NaN,...,1996-09-18,0.0,50.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Wings of Courage,False,6.8,4.0


In [9]:
(metadata[metadata['release_date'] > '2009-01-01'])

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
868,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,79782,tt1684935,en,Wenecja,An atmospheric coming-of-age story featuring a...,...,2010-05-25,0.0,110.0,"[{'iso_639_1': 'pl', 'name': 'Polski'}]",Released,NaN,Venice,False,7.5,4.0
1081,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,141210,tt2250194,en,The Sleepover,"The town of Derry has a secret, but no one tol...",...,2013-10-12,0.0,6.0,[],Released,NaN,The Sleepover,False,8.0,1.0
2114,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",http://www.thefarmerswifefilm.co.uk/,143750,tt2140519,en,The Farmer's Wife,"As her surroundings are invaded by outsiders, ...",...,2012-06-20,0.0,18.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Farmer's Wife,False,10.0,1.0
2564,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,tt1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,2012-03-22,0.0,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Nation. Underfed.,A Place at the Table,False,6.9,7.0
2778,False,NaN,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",NaN,171982,tt2378428,en,Romance,She's as hot as Britney Spears. Hotter. She pa...,...,2012-10-09,0.0,27.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Romance,False,0.0,0.0
4122,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",http://brokenkingdomfilm.com/#/kingdomcome,136558,tt2043893,en,Kingdom Come,"The documentary, Kingdom Come follows a first-...",...,2011-01-01,0.0,88.0,[],Released,The unmaking of independent film,Kingdom Come,False,8.0,1.0
4356,False,NaN,3512454,"[{'id': 18, 'name': 'Drama'}]",NaN,110428,tt2018086,fr,Camille Claudel 1915,"Winter, 1915. Confined by her family to an asy...",...,2013-03-13,115860.0,95.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Camille Claudel 1915,False,7.0,20.0
5766,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,78022,tt1790669,en,Da wu sheng,"In the 19th century, the Prince Regent of the ...",...,2011-09-08,0.0,95.0,"[{'iso_639_1': 'zh', 'name': '普通话'}, {'iso_639...",Released,NaN,My Kingdom,False,4.0,2.0
6563,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,66111,tt1351186,en,Luster,Thomas Luster is a troubled businessman who tr...,...,2010-12-12,0.0,0.0,[],Rumored,Know Your Enemy,Luster,False,4.8,4.0
11155,False,NaN,40000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",NaN,77221,tt1701210,en,Black Gold,"On the Arabian Peninsula in the 1930s, two war...",...,2011-12-21,5446000.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Black Gold,False,5.9,77.0


In [132]:
movies_2017_list = list(movies.find())

In [133]:
movies_2017_df = pd.DataFrame(movies_2017_list)

In [135]:
movies_2017_df.to_csv('data/movies_2017_html.csv')